Visualization 3 (Enhanced): CAR Distribution across All Windows

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

# --- Visualization 3 (Enhanced): CAR Distribution across All Windows ---

# Load the wide-format results
try:
    df_wide = pd.read_csv("./outcome/event_study_wide_results.csv")
except FileNotFoundError:
    print("Error: 'event_study_wide_results.csv' not found in the 'outcome' folder.")
    exit()

# --- Data Preparation ---
# Melt the DataFrame to a long format
id_vars = ['Asset', 'EventGroup', 'EventDate']
value_vars = [col for col in df_wide.columns if col.startswith('CAR_')]
df_long = pd.melt(df_wide, id_vars=id_vars, value_vars=value_vars,
                  var_name='CAR_Type', value_name='CAR_Value')

# Extract Model and Window from the 'CAR_Type' column
def parse_car_type(car_type_str):
    match = re.search(r'CAR_(.*)\((.*)\)', car_type_str)
    if match: return match.group(1), f"({match.group(2)})"
    return None, None
df_long[['Model', 'Window']] = df_long['CAR_Type'].apply(lambda x: pd.Series(parse_car_type(x)))

# Filter for the specific model we want to visualize
model_to_plot = 'MM_SPY'
df_plot_data = df_long[df_long['Model'] == model_to_plot]

# Define order for facets
window_order = ['(-1,+1)', '(-5,+5)', '(-10,+10)']


# --- Plotting ---
# Use catplot to create a grid of violin plots
g = sns.catplot(
    data=df_plot_data,
    x='EventGroup',
    y='CAR_Value',
    kind='violin',
    row='Asset',          # Each row is an Asset
    col='Window',         # Each column is an Event Window
    col_order=window_order,
    palette='muted',
    inner='quart',
    height=4,
    aspect=1.8,
    sharey=True           # Use the same y-axis for all subplots for easy comparison
)

# --- Chart Enhancement ---
g.fig.suptitle(f'Distribution of CARs by Asset and Event Window\nModel: {model_to_plot}', y=1.03)
g.set_axis_labels("Event Group", "CAR Value")
g.set_titles(row_template="Asset: {row_name}", col_template="Window: {col_name}")
g.despine(left=True)

# Add a horizontal line at y=0
for ax in g.axes.flat:
    ax.axhline(0, ls='--', color='black', linewidth=0.8)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

Visualization 4: Cumulative Distribution Function (CDF) Plot

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

# --- Visualization 4: Cumulative Distribution Function (CDF) Plot ---

# Load the wide-format results which contain individual event CARs
try:
    df_wide = pd.read_csv("./outcome/event_study_wide_results.csv")
except FileNotFoundError:
    print("Error: 'event_study_wide_results.csv' not found in the 'outcome' folder.")
    exit()

# --- Data Preparation ---
# Melt the DataFrame to a 'long' format, same as for the violin plot
id_vars = ['Asset', 'EventGroup', 'EventDate']
value_vars = [col for col in df_wide.columns if col.startswith('CAR_')]
df_long = pd.melt(df_wide, id_vars=id_vars, value_vars=value_vars,
                  var_name='CAR_Type', value_name='CAR_Value')

# Extract Model and Window from the 'CAR_Type' column
def parse_car_type(car_type_str):
    match = re.search(r'CAR_(.*)\((.*)\)', car_type_str)
    if match: return match.group(1), f"({match.group(2)})"
    return None, None
df_long[['Model', 'Window']] = df_long['CAR_Type'].apply(lambda x: pd.Series(parse_car_type(x)))

# --- Plotting ---
# To create a clear comparison, let's focus on a single asset, model, and window.
# We will compare the CAR distributions for different event groups.
asset_to_plot = 'Bitcoin'
model_to_plot = 'MM_SPY'
window_to_plot = '(-10,+10)'

# Filter the data for our specific slice
df_plot_data = df_long[
    (df_long['Asset'] == asset_to_plot) &
    (df_long['Model'] == model_to_plot) &
    (df_long['Window'] == window_to_plot)
]

# Let's compare key training groups
groups_to_compare = ['internal_good_train', 'internal_bad_train', 'external_bad_train']
df_plot_data = df_plot_data[df_plot_data['EventGroup'].isin(groups_to_compare)]


plt.figure(figsize=(12, 7))
sns.ecdfplot(data=df_plot_data, x='CAR_Value', hue='EventGroup', linewidth=2.5)

# --- Chart Enhancement ---
plt.title(f'Cumulative Distribution of CARs for {asset_to_plot}\nModel: {model_to_plot}, Window: {window_to_plot}', fontsize=16)
plt.xlabel('CAR Value', fontsize=12)
plt.ylabel('Cumulative Probability', fontsize=12)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.axvline(0, ls=':', color='black', linewidth=1) # Vertical line at x=0
plt.legend(title='Event Group')

plt.show()